In [ ]:
import pandas as pd
import numpy as np
import pickle as pkl
import glob
import re
import requests
import random
import time

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', 1000)

In [ ]:
# format possible incident id directory into a list
incident_file_list = [filename.split('incident/')[1] for filename in glob.glob('../gun_violence_archive_spider/www.gunviolencearchive.org/incident/*')]

In [ ]:
# format all possible_ids from std_out file into a list
df = pd.read_csv('../Data Sources/std_out.csv', encoding='latin-1', engine='python', index_col='Unnamed: 0')
df.columns = ['std_out']
incident_std_list = list(df[(['gunviolencearchive.org/incident/' in s for s in df['std_out']])]['std_out'])
incident_std_list = [i.split('/incident/')[1] for i in incident_std_list]
incident_std_list = [i.split('\x90')[0] for i in incident_std_list]
incident_std_list = [re.sub('[^0-9]','', i) for i in incident_std_list]
incident_std_list = list(set(incident_std_list))

In [ ]:
# format all ids that have already been catalogued by pandas into a list
df = pd.read_pickle('../Pickles/df1.pkl')
logged_ids_list = sorted(list(df['incident_id'].unique()))

In [ ]:
# making sure every catalogued (scraped and downloaded) ID is in the incident folder,
# so when requests are made it will only be for non-catalogued IDs
check_list = logged_ids_list + incident_std_list
check_list = set(list(check_list))
add_list = []
for incident_id in check_list:
    if '/' in str(incident_id):
        incident_id = incident_id.split('/')[0]
    if str(incident_id) in incident_file_list:
        pass
    else:
        add_list.append(int(incident_id))

with open('../gun_violence_archive_spider/www.gunviolencearchive.org/incident/id_add_list.txt', 'w') as f:
    for item in add_list:
        f.write("%s\n" % item)
        
print(len(add_list))

# command to create list of files from id_add_list.txt
# head to the incident directory and then run the following command
# cat id_add_list.txt | xargs touch

In [ ]:
# rerun creation of list because new items may have been added
# format possible incident id directory into a list
# catalogue all possible incident_ids into a csv file
incident_file_list = [filename.split('incident/')[1] for filename in glob.glob('../gun_violence_archive_spider/www.gunviolencearchive.org/incident/*')]
df = pd.DataFrame(incident_file_list, columns = ['possible_incident_ids.csv'])
df.to_csv('../Data Sources/possible_incident_ids.csv')

In [ ]:
# rerun creation of list because new items may have been added
# format all ids that have already been catalogued by pandas into a list
df = pd.read_pickle('../Pickles/df1.pkl')
logged_ids_list = sorted(list(df['incident_id'].unique()))

In [ ]:
# range from lowest actual id to highest actual id
# lets check for ids that may have been missed when scraping ids
# these are all possible numbers from smallest id to largest id
# if the possible id is not in the possible_incident_id list
# known_possible_ids is all ids that have been logged or recorded so far
known_possible_ids = incident_file_list + logged_ids_list
known_possible_ids = list(set(known_possible_ids))
# check for what's already been added to the text file previously
already_appended_df = pd.read_csv('../Data Sources/brute_force_id_status_codes.txt')
already_appended_df.columns=['id', 'status_code']
already_appended_df['id'] = already_appended_df['id'].astype(int)
already_appended_df['status_code'] = already_appended_df['status_code'].astype(int)
already_appended_list = list(already_appended_df['id'].unique())

for possible_id in np.arange(478855, 1350888+1):
    if str(possible_id) in known_possible_ids:
        pass
    elif possible_id in already_appended_list and already_appended_df.loc[already_appended_df['id']==possible_id, 'status_code'].values[0]!=404 and already_appended_df.loc[already_appended_df['id']==possible_id, 'status_code'].values[0]!=200:
        pass
    else:
        with open('../Data Sources/brute_force_id_status_codes.txt', 'a+') as f:
            result = requests.get("https://www.gunviolencearchive.org/incident/" + str(possible_id))
            f.write("\n" + str(possible_id) + ', ' + str(result.status_code))
            print(str(possible_id) + ', ' + str(result.status_code))
            time.sleep(2.5 + random.uniform(0.5, 2))

In [ ]:
# iterating over ids in incident folder
# if it's not in the dataframe, then adding it to the scraper queue
id_scrape_list = []
for incident_id in incident_file_list:
    if '/' in incident_id:
        incident_id = incident_id.split('/')[0]
    if incident_id=='id_add_list.txt':
        pass
    elif int(incident_id) in logged_ids_list:
        pass
    else:
        id_scrape_list.append(int(incident_id))

with open('../Pickles/incident_id_scrape_queue.pkl', 'wb') as f:
    pkl.dump(id_scrape_list, f)
    
print(len(id_scrape_list))

In [ ]:
# exporting df_date un_scaled data to assets repository for d3.js visualization
df_date = pd.read_pickle('../Pickles/df_date_2.pkl')
df_date = df_date[['date', 'num_victims']]
df_date.to_csv('../../cyaris.github.io/assets/line_graph_data.csv')